In [77]:
import numpy as np
import math
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import timeit

In [78]:
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

# 1.B

In [110]:
param_dict={
    "W1": torch.randn((13, 20), requires_grad=True),
    "W2": torch.randn((20, 1), requires_grad=True),
    "B1": torch.randn((20), requires_grad=True),
    "B2": torch.randn((1), requires_grad=True)
       }

In [111]:
## Define the network
def my_nn(input,param_dict): 
    W1=param_dict["W1"]
    W2=param_dict["W2"]
    B1=param_dict["B1"]
    B2=param_dict["B2"]
    h1 = torch.tanh((input @ W1) + B1)
    output = h1 @ W2 + B2
    return F.log_softmax(output.reshape(-1,1), dim=1)



In [112]:
example_data = torch.randn(( 20,13))
example_data

tensor([[ 0.4194, -0.5635, -0.7265, -0.8678,  1.3517, -0.6921, -1.2241, -0.1145,
          0.3294, -0.2985,  1.5992, -0.6556,  1.2602],
        [ 0.6317, -2.1409, -0.0872,  0.0149, -0.4793, -0.2583,  1.8375, -1.5414,
         -0.3280,  1.0957, -0.6149,  0.1456,  0.2219],
        [-0.3978,  0.1971, -0.3978, -1.1510, -0.1578,  1.6268,  0.1436,  0.7801,
         -0.0717,  0.7367,  1.0798,  1.0371, -1.4322],
        [ 0.0347,  0.0403, -0.6927, -1.2089, -0.1132, -0.9543,  1.4943,  1.7300,
         -0.2849,  0.7907,  0.1384,  0.9055,  0.4831],
        [ 1.4759,  0.0654, -1.9430, -0.3932,  1.0899, -0.3019, -1.6342, -2.2375,
          0.9851,  0.5468, -0.4235, -0.6607, -1.7647],
        [-0.8693,  0.8092,  1.8604,  0.7818, -0.3790, -0.7088,  0.1125, -0.5216,
          1.0001, -2.3189,  0.7714,  2.1244, -0.7423],
        [-0.9377, -1.1109, -0.4058,  0.4425,  0.3863,  0.8178, -1.6946,  1.1890,
          0.7923,  1.4628,  0.7178,  0.9744, -0.8725],
        [-1.3917, -0.2024, -0.3276,  1.1504,  0.

In [131]:
start = timeit.default_timer()  
y=my_nn(example_data,param_dict)
print(y.shape)
stop = timeit.default_timer()
print('FF Time= :',stop - start) 

torch.Size([20, 1])
FF Time= : 0.004650638000384788


In [132]:
loss_fn = torch.nn.NLLLoss() 

In [152]:
from sklearn.datasets import load_boston
X, y = load_boston(return_X_y=True)
y=torch.as_tensor(y).long()
print(y.size())

start = timeit.default_timer()  
for i in range(15):
    y_pred=my_nn(torch.as_tensor(X).float(),param_dict)
    print(y_pred[0].size())
    loss = loss_fn(y_pred, y)
    my_nn.zero_grad()
    loss.backward()
#     print(y.shape)
stop = timeit.default_timer()
print('FF Time= :',stop - start) 

torch.Size([506])
torch.Size([1])


IndexError: Target 24 is out of bounds.

In [ ]:

for i in range(50):
    y_pred = model(X_trainTensor)
    loss = loss_fn(y_pred, y_trainTensor)
    model.zero_grad()
    loss.backward()
    with torch.no_grad():       
        for param in model.parameters():
            param -= learning_rate * param.grad

## 2.a

In [73]:
def exp_reducer(w,x):
    return torch.tanh_(w.mm(x))
import torch 
Tensors={"W1": torch.rand(30, 2, requires_grad=True),
"W2" : torch.rand(30, 30, requires_grad=True),
"W3" : torch.rand(10, 30, requires_grad=True),
"X":  torch.rand(2, 1, requires_grad=True)
}
y=torch.autograd.functional.jacobian(exp_reducer,(Tensors["W1"], Tensors["X"]),create_graph=True)
y=torch.autograd.functional.jacobian(exp_reducer,(Tensors["W2"], y[0][0][0]),create_graph=True)
y=torch.autograd.functional.jacobian(exp_reducer,(Tensors["W3"], y[0][0][0]),create_graph=True)
print(y[1].shape)

torch.Size([10, 30, 30, 30])


In [75]:
### import torch 
Tensors={"W1": torch.rand(30, 2, requires_grad=True),
"W2" : torch.rand(30, 30, requires_grad=True),
"W3" : torch.rand(10, 30, requires_grad=True),
"X":  torch.rand(2, 1, requires_grad=True)
}
# Tensors["X"].zero_grad()
y=torch.tanh_(Tensors["W1"].mm(Tensors["X"]))
# print(y)
y.sum().backward(retain_graph=True)
y=torch.tanh_(Tensors["W2"].mm(y))
y.sum().backward(retain_graph=True)
y=torch.tanh_(Tensors["W3"].mm(y))
y.sum().backward(retain_graph=True)
print("inp.grad")
print(Tensors["X"].grad)

inp.grad
tensor([[6.1222],
        [6.7758]])


## 2.e

Forward Mode AutoDif:
M^3 + M^2 Ops =O(M^3)
<br>Reverse Mode AutoDiff (Backprop):
M^2 + M^2 Ops= O(M^2)
<br>• Finite difference requires 2*D forward passes, with D parameters
<br>• Reverse Mode AD, often ~ 2x forward pass
<br>• Forward Mode AD speed / forward pass would increase with width

# 4.a

In [79]:
class MyModule(nn.Module):
    def __init__(self,num_layers):
        super(MyModule, self).__init__()
        self.linears = nn.ModuleList([nn.Linear(10, 10) for i in range(num_layers)])

    def forward(self, x):
        # ModuleList can act as an iterable, or be indexed using ints
        for i, l in enumerate(self.linears):
            x = self.linears[i // 2](x) + l(x)
        return x

## 4.b

In [247]:
def xavier(ni,no):
    return np.random.randn(ni,no)*np.math.sqrt(6/(ni+no))
xavier(100,100)

def weights_init_uniform(m,d):
    classname = m.__class__.__name__
    # for every Linear layer in a model..
    for layer in model.linears:
        # apply a uniform distribution to the weights and a bias=0
        layer.weight.data.uniform_(-1*d, d)
        layer.bias.data.fill_(0)
    return m

In [248]:
batch_size_train=32
batch_size_test=32
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)
print(example_data.shape)

torch.Size([32, 1, 28, 28])


## 4.c

In [253]:
ds=[0.01,0.1,2]
loss=torch.nn.CrossEntropyLoss()
for d in ds:    
    model = my_model(8)
    model=weights_init_uniform(model,d)
    model.train()
#     print(model)
#     print(model.linears[1].weight.data)
#     print(model.linears[1].bias.data)
    for batch_idx, (data, target) in enumerate(train_loader):
#         optimizer.zero_grad()
        print(data.shape)
        output = model(data).mean()
        print(output)
        print(target)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item()))
            train_losses.append(loss.item())
            train_counter.append(
            (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
            torch.save(network.state_dict(), '/results/model.pth')
            torch.save(optimizer.state_dict(), '/results/optimizer.pth')
        
#     start = timeit.default_timer()  
#     loss_fn = torch.nn.MSELoss(reduction='sum')
#     for idx in range(len(example_data)):
#         out=model(example_data[idx][0]).mean()
#         print(idx,out)
#         out.backward()
#     stop = timeit.default_timer()
#     print('FF 256 images Time= :',stop - start) 

torch.Size([32, 1, 28, 28])
tensor(2.3653e-12, grad_fn=<MeanBackward0>)
tensor([8, 8, 8, 6, 2, 7, 6, 1, 6, 7, 1, 0, 3, 7, 9, 0, 0, 1, 8, 1, 7, 0, 8, 5,
        4, 4, 6, 1, 3, 2, 1, 7])


ValueError: Expected 2 or more dimensions (got 0)

In [156]:
a=MyModule()
a.forward()

TypeError: forward() missing 1 required positional argument: 'x'